# a bare ☠ approach to c🍪🍪kiecutter

[Cookiecutter](https://github.com/audreyr/cookiecutter) rules.  I basically seems like jinja templates of files names and files.  Let's build that.

In [1]:
from nbconvert import *
from toolz.curried import *

Use the `nbconvert` jinja environment.

In [2]:
e = exporters.templateexporter.TemplateExporter()
template = compose(e.environment.from_string, str)
render =  lambda str, **kw: template(str).render(**kw)

Set some reasonable defaults to ticker with.

In [3]:
from pathlib import Path 

top, to = Path('template'), Path('template/projectA')
ignores = 'checkpoint', 'diffs', str(to), '.DS_Store', '.git'
nm = 'Tony Fast'
data = dict(name=to.name,author=dict(name=nm, email='.'.join(map(str.lower, nm.split()))+'@gmail.com', user=''.join(map(str.lower, nm.split()))))

## Scanner

Create a `Scanner` project `name` from `source` ignorning files matching `ignores`.

`Scatter.template` templates all `jinja` templates in the files names and files writing to `dir`.

In [4]:
class Scanner(object):
    def __init__(self, name, source='.', ignores=ignores, **data):
        self.name, self.source, self.data, self.ignores = name, Path(source), data, ignores
        
    def template(self, dir='..', **data):
        from json import loads, dumps
        data = {**data, 'name': self.name}
        for path in self:
            target = Path(render(dir / path.relative_to(self.source), **data))
            
            print('Creating {}'.format(target)) or create_parents(target) 
            
            path.is_file() and pipe(
                path, Path.read_text, 
                path.suffix in {'.ipynb', '.json'} and 
                compose(dumps, partial(dispatchjson, **data), loads) 
                or partial(render, **data), target.write_text)
                    
    def __iter__(self):
        for _ in self.source.rglob('*'):
            if any(ignore in str(_) for ignore in self.ignores): continue
            yield _

## Utility Functions

In [5]:
def create_parents(path, paths=[]):    
    """Make sure directions exist."""
    while not path.parent.exists(): path = paths.append(path.parent) or path.parent
    while paths: paths.pop(-1).mkdir()

def dispatchjson(object, **data):
    """Tempplate json files."""
    if type(object) is list: 
        return list(dispatchjson(_, **data) for _ in object)
    if type(object) is dict:
        return {dispatchjson(k, **data): dispatchjson(v, **data) for k, v in object.items()}
    if type(object) is str: object = render(object, **data)
    return object

In [6]:
if __name__ == '__main__':
    !git clone https://github.com/tonyfast/template
    %rm -rf template/projectB
    Scanner('projectB', top).template('template/projectB', description='Demo', **data)

fatal: destination path 'template' already exists and is not an empty directory.
Creating template/projectB/config.ipynb
Creating template/projectB/readme.ipynb
Creating template/projectB/requirements.txt
Creating template/projectB/setup.py
Creating template/projectB/projectB
Creating template/projectB/projectB/__init__.py
Creating template/projectB/projectB/projectB.ipynb
